In [1]:
# 导包
import torch
import os
import argparse
import pandas as pd
import tqdm
import numpy as np
# os.environ['NUMEXPR_MAX_THREADS'] = '48'
# 导入各个指标
import sys
sys.path.append('/home/dengruijun/data/FinTech/PP-Split/')
# utils
from ppsplit.utils.utils import create_dir

/home/dengruijun/miniconda3/envs/drj-pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 导入各个baseline模型及其数据集预处理方法
# 模型
from target_model.models.splitnn_utils import split_weights_client
from target_model.models.VGG import VGG,VGG5Decoder,model_cfg
from target_model.models.BankNet import BankNet1,BankNetDecoder1,bank_cfg
from target_model.models.CreditNet import CreditNet1,CreditNetDecoder1,credit_cfg
from target_model.models.PurchaseNet import PurchaseClassifier1,PurchaseDecoder1,purchase_cfg
from target_model.models.IrisNet import IrisNet,IrisNetDecoder,Iris_cfg
from target_model.models.ResNet import resnet18,resnet34,resnet50,InversionNet,resnet_model_cfg
from target_model.models.AlexNet import AlexNet_MNIST


# 数据预处理方法
from target_model.data_preprocessing.preprocess_cifar10 import get_cifar10_normalize,deprocess
from target_model.data_preprocessing.preprocess_bank import bank_dataset,preprocess_bank,preprocess_bank_dataset,tabinfo_bank
from target_model.data_preprocessing.preprocess_credit import preprocess_credit,tabinfo_credit
from target_model.data_preprocessing.preprocess_purchase import preprocess_purchase,tabinfo_purchase
from target_model.data_preprocessing.preprocess_Iris import preprocess_Iris, tabinfo_Iris
from target_model.data_preprocessing.dataset import get_one_data

# utils
from  ppsplit.utils.utils import create_dir


# model_select
from target_model.task_select import get_dataloader_and_model,get_dataloader_and_model, \
    get_dataloader,get_models,get_infotopo_para

In [3]:

args = {
        'device':torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
        # 'device':torch.device("cpu"),
        # 'dataset':'CIFAR10',
        # 'dataset':'bank',
        # 'dataset':'credit',
        'dataset':'purchase',
        # 'dataset':'Iris',
        # 'model': 'ResNet18',
        # 'model': 'VGG5',
        'model': 'VGG9',
        # 'result_dir': '20240702-FIL/',
        'result_dir': '20240702-effectiveInfo/',
        'oneData_bs': 500,
        'test_bs': 500,
        'train_bs': 1,
        'noise_scale': 0, # 防护措施
        'split_layer': 5,
        # 'test_num': 'invdFIL', # MI, invdFIL, distCor, ULoss,  # split layer [2,3,5,7,9,11] for ResNet18
        'test_num': 'effectiveInfo1.8.1',
        'no_dense':True,
        }
print(args['device'])
print(args)

cuda:0
{'device': device(type='cuda', index=0), 'dataset': 'purchase', 'model': 'VGG9', 'result_dir': '20240702-effectiveInfo/', 'oneData_bs': 500, 'test_bs': 500, 'train_bs': 1, 'noise_scale': 0, 'split_layer': 5, 'test_num': 'effectiveInfo1.8.1', 'no_dense': True}


# 1. ResNet18

In [4]:
# resnet18
net = resnet18(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='gelu', pooling='avg')
print(net)

Num unit layers: 14
Split layer: -1
ResNet(
  (selected_layers): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate=none)
    (3): AvgPool2d(kernel_size=3, stride=2, padding=1)
    (4): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): GELU(approximate=none)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (5): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): GELU

In [ ]:
# decoder
# inet = InversionNet(in_c=64, upconv_channels=[(128, 'same'), (3, 'same')], last_activation=None)
inet = InversionNet(last_activation=None)
print(inet)

In [7]:
# 加载一下模型参数
# resnet18 cifar10
# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/CIFAR10-models/ResNet18/32bs-ep20-relu-max-adam/resnet18-drj.pth' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# client_net = resnet18(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='gelu', pooling='avg')

# resnet18 cifar100

# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18/CIFAR100/resnet18-drj.pth.ckpt' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18/CIFAR100/resnet18-drj.pth'
# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18/CIFAR100/resnet18-drj-align.pth'
# client_net = resnet18(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=100, activation='gelu', pooling='avg')

# resnet34 cifar100
# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet34/CIFAR100/resnet34-drj.pth.ckpt' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet34/CIFAR100/resnet34-drj.pth'
# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet34/CIFAR100/resnet34-drj-align.pth'
# client_net = resnet34(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=100, activation='gelu', pooling='avg')

# resnet50 cifar100
# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet50/CIFAR100/resnet50-drj.pth.ckpt' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet50/CIFAR100/resnet50-drj.pth'
# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet50/CIFAR100/resnet50-drj-align.pth'
# # client_net = resnet50(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=100, activation='gelu', pooling='avg')

# resnet18 20ep cifar10
# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18/CIFAR10/epoch=14resnet18-drj.pth.ckpt' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18/CIFAR10/resnet18-drj.pth'
# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18/CIFAR10/resnet18-drj-align.pth'
# client_net = resnet18(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='gelu', pooling='avg')


# resnet18 20ep narrow cifar10
# unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_narrow/CIFAR10/epoch=16resnet18_narrow-drj.pth.ckpt' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
# new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_narrow/CIFAR10/resnet18-drj.pth'
# new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_narrow/CIFAR10/resnet18-drj-align.pth'
# client_net = resnet18(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='gelu', pooling='avg')

# resnett18 20ep wide cifar10
unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_wide/CIFAR10/epoch=14resnet18_wide-drj.pth.ckpt' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
new_unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_wide/CIFAR10/resnet18-drj.pth'
new_unit_net_route_align = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/ResNet/resnet18_wide/CIFAR10/resnet18-drj-align.pth'
client_net = resnet18(pretrained=False, split_layer=-1, bottleneck_dim=-1, num_classes=10, activation='gelu', pooling='avg')


# 提取state_dict
pweights = torch.load(unit_net_route)
print(pweights.keys())
state_dict = pweights['state_dict']
torch.save(state_dict, new_unit_net_route)

# 修改key


Num unit layers: 14
Split layer: -1
dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters', 'datamodule_hparams_name', 'datamodule_hyper_parameters'])


In [8]:
# 改一下 resnet的 模型参数的key
import sys
sys.path.append('/home/dengruijun/data/FinTech/PP-Split/')
import torch

# from target_model.task_select import *

pweights = torch.load(new_unit_net_route)
cweights = client_net.state_dict()

print(len(pweights.keys()),pweights.keys())
print(len(cweights.keys()),cweights.keys())


# new_key = {}
# for keyp,keyc in zip(pweights.keys(),cweights.keys()):
#     print(keyp,'\t\t\t',keyc)

# for i,key in enumerate(pweights.keys()):
#     if i<122:
#         continue
#     print(key)

    # new_key[key.replace('model.','')] = pweights[key]

new_key = {}
for key in cweights.keys():
    # new_key[key] = pweights[key.replace('selected_layers','model.layers')] # resnet18 cifar10
    # new_key[key] = pweights[key.replace('selected_layers','model.selected_layers')] # resnet18 cifar100
    new_key[key] = pweights[key.replace('selected_layers','model.selected_layers')] # resnet18 cifar100
torch.save(new_key, new_unit_net_route_align)

122 odict_keys(['model.selected_layers.0.weight', 'model.selected_layers.1.weight', 'model.selected_layers.1.bias', 'model.selected_layers.1.running_mean', 'model.selected_layers.1.running_var', 'model.selected_layers.1.num_batches_tracked', 'model.selected_layers.4.conv1.weight', 'model.selected_layers.4.bn1.weight', 'model.selected_layers.4.bn1.bias', 'model.selected_layers.4.bn1.running_mean', 'model.selected_layers.4.bn1.running_var', 'model.selected_layers.4.bn1.num_batches_tracked', 'model.selected_layers.4.conv2.weight', 'model.selected_layers.4.bn2.weight', 'model.selected_layers.4.bn2.bias', 'model.selected_layers.4.bn2.running_mean', 'model.selected_layers.4.bn2.running_var', 'model.selected_layers.4.bn2.num_batches_tracked', 'model.selected_layers.5.conv1.weight', 'model.selected_layers.5.bn1.weight', 'model.selected_layers.5.bn1.bias', 'model.selected_layers.5.bn1.running_mean', 'model.selected_layers.5.bn1.running_var', 'model.selected_layers.5.bn1.num_batches_tracked', 'm

In [9]:
# 加载模型参数
# client_net = resnet18(pretrained=False, split_layer=13, bottleneck_dim=-1, num_classes=100, activation='gelu', pooling='avg')
state_dict = torch.load(new_unit_net_route_align)
client_net.load_state_dict(state_dict) 
print(client_net)

ResNet(
  (selected_layers): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate=none)
    (3): AvgPool2d(kernel_size=3, stride=2, padding=1)
    (4): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): GELU(approximate=none)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (5): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): GELU(approximate=none)
      (conv2): Co

# 2. AlexNet

In [ ]:
# alexnet mnist
net = AlexNet_MNIST()
print(net)
# print(len(net)) 

# 3. VGG9

In [ ]:
# model_msg = get_models(args)
# # 模型
# client_net,decoder_net = model_msg['client_net'],model_msg['decoder_net']
# decoder_route = model_msg['decoder_route']
# image_deprocess = model_msg['image_deprocess']


client_net = VGG('Unit','VGG9',len(model_cfg['VGG9'])-1,model_cfg,noise_scale=args['noise_scale'])
print(client_net)

In [ ]:
client_net = VGG('Unit','VGG5',len(model_cfg['VGG5'])-1,model_cfg,noise_scale=args['noise_scale'])
print(client_net)